## from Saksham
The goal is to create a PlaneSegmentation object while iteratively writing 'image_mask'. <br>
Steps I followed:<br>
1. Created an image_mask (3x3) numpy array.
2. Pass that into a custom generator class iter_datasetvieww that iterates over the rows. 
3. Wrap that generator with a DataChunkIterator and then either use add_roi method to populate the plane segmentation OR create a VectorData with all the image_masks concatenated in the first dimension.

In [2]:
from pynwb import NWBFile, NWBHDF5IO
from pynwb.ophys import TwoPhotonSeries, OpticalChannel, ImageSegmentation, Fluorescence
from pynwb.device import Device
from hdmf.data_utils import DataChunkIterator
from hdmf.common.table import VectorData
from hdmf.backends.hdf5.h5_utils import H5DataIO
from datetime import datetime
from dateutil.tz import tzlocal
import numpy as np

In [3]:
def iter_datasetvieww(datasetview_obj):
    '''
    Generator to return a row of the array each time it is called.
    This will be wrapped with a DataChunkIterator class.

    Parameters
    ----------
    datasetview_obj: DatasetView
        2-D array to iteratively write to nwb.
    '''

    for i in range(datasetview_obj.shape[0]):
        curr_data = datasetview_obj[i]
        yield curr_data
    return


class MyDataChunkIterator(DataChunkIterator):

    def __init__(self, gen_obj, dataset_obj):
        super().__init__(gen_obj)
        self.a = dataset_obj

    def __len__(self):
        return self.a.shape[0]

In [4]:
# Create nwbfile
nwbfile = NWBFile('my first synthetic recording', 'EXAMPLE_ID', datetime.now(tzlocal()),
                  experimenter='Dr. Bilbo Baggins',
                  lab='Bag End Laboratory',
                  institution='University of Middle Earth at the Shire',
                  experiment_description=('I went on an adventure with thirteen '
                                          'dwarves to reclaim vast treasures.'),
                  session_id='LONELYMTN')

# Device
device = Device('imaging_device_1')
nwbfile.add_device(device)

# Optical Channels
op_names = ['my_optchan1', 'my_optchan2']
optical_channel1 = OpticalChannel('my_optchan1', 'description1', 500.)
optical_channel2 = OpticalChannel('my_optchan2', 'description2', 600.)

# Imaging Plane
imaging_plane = nwbfile.create_imaging_plane(
    name = 'my_imgpln', 
    optical_channel = [optical_channel1, optical_channel2], 
    description='a very interesting part of the brain',
    device=device, 
    excitation_lambda=600., 
    imaging_rate=300., 
    indicator='GFP', 
    location='my favorite brain location',
    manifold=np.ones((5, 5, 3)), 
    conversion=4.0, 
    unit='manifold unit', 
    reference_frame='A frame to refer to'
)

# TwoPhotonSeries
image_series = TwoPhotonSeries(
    name='test_iS', 
    dimension=[2],
    external_file=['images.tiff'], 
    imaging_plane=imaging_plane,
    starting_frame=[0], 
    format='tiff', 
    starting_time=0.0, 
    rate=1.0
)
nwbfile.add_acquisition(image_series)

# Processing module / ImageSegmentation
mod = nwbfile.create_processing_module('ophys', 'contains optical physiology processed data')
img_seg = ImageSegmentation()
mod.add(img_seg)
ps = img_seg.create_plane_segmentation(
    description='output from segmenting my favorite imaging plane',
    imaging_plane=imaging_plane, 
    name='my_planeseg', 
    reference_images=image_series
)

In [5]:
# APPROACH 1
w, h = 3, 3
pix_mask1 = [(0, 0, 1.1), (1, 1, 1.2), (2, 2, 1.3)]
img_mask1 = [[0.0 for x in range(w)] for y in range(h)]
# each image is placed in 1 row, with first dim of image shifted to the second of the stored matrix
img_mask1[0][0] = 1.1
img_mask1[1][1] = 1.2
img_mask1[2][2] = 1.3
img_mask1[1][2] = 2.1
print(f'Image mask 1: {img_mask1}')

# pix_mask_generator = DataChunkIterator(iter_datasetvieww(np.array(pix_mask1)))
img_mask_generator1 = MyDataChunkIterator(iter_datasetvieww(np.array(img_mask1)), np.array(img_mask1))

# ps.add_roi(pixel_mask=pix_mask_generator, image_mask=img_mask_generator, id=4)
ps.add_roi(image_mask=img_mask_generator1, id=4)
# ps.add_roi(pixel_mask=pix_mask1, image_mask=img_mask1, id=4)
# ps.add_roi(image_mask=img_mask1, id=4)

pix_mask2 = [(0, 0, 2.1), (1, 1, 2.2)]
img_mask2 = [[0.0 for x in range(w)] for y in range(h)]
img_mask2[0][0] = 2.1
img_mask2[1][1] = 2.2
# pix_mask_generator2 = DataChunkIterator(iter_datasetvieww(np.array(pix_mask2)))
img_mask_generator2 = MyDataChunkIterator(iter_datasetvieww(np.array(img_mask2)), np.array(img_mask1))
# ps.add_roi(pixel_mask=pix_mask_generator2, image_mask=img_mask_generator2, id=9)
ps.add_roi(image_mask=img_mask_generator2, id=9)
# ps.add_roi(pixel_mask=pix_mask2, image_mask=img_mask2, id=9)
# ps.add_roi(image_mask=img_mask2, id=9)

# ----------------------------------------------------------------------------------
# APPROACH 2:
# img_mask_gen = MyDataChunkIterator(iter_datasetvieww(np.array([img_mask1, img_mask2])), np.array([img_mask1, img_mask2]))
# vd1_custom = VectorData('image_mask', 'desc', img_mask_gen)
# vd2_custom = VectorData('pixel_mask', 'desc', [None])
# vd2_custom = VectorData('pixel_mask_id', 'desc', [None])
# ps2 = img_seg.create_plane_segmentation('output', imaging_plane, 'my_planeseg2', image_series, columns=[vd1_custom])

# ps2.add_column('image_mask', 'desc', data=img_mask_gen)
# ps2.add_column('image_mask', 'desc', data=H5DataIO(np.array([img_mask1, img_mask2]), compression=True))
# print(vd1_custom.data[:])

# ==================================================================================

Image mask 1: [[1.1, 0.0, 0.0], [0.0, 1.2, 2.1], [0.0, 0.0, 1.3]]


In [6]:
with NWBHDF5IO('ophys_example_test.nwb', 'w') as io:
    io.write(nwbfile)
io = NWBHDF5IO('ophys_example_test.nwb', 'r')
nwbfile = io.read()
io.close()

TypeError: 'MyDataChunkIterator' object does not support indexing

In [4]:
# Fluorescence
fl = Fluorescence()
mod.add(fl)

rt_region = ps.create_roi_table_region('the first of two ROIs', region=[0, 1])
rt_region1 = ps.create_roi_table_region('the second of two ROIs', region=[1])
####################
# Now that you have an :py:class:`~pynwb.core.DynamicTableRegion`, you can create your an
# :py:class:`~pynwb.ophys.RoiResponseSeries`.

dat1 = np.random.randint(20, size=[3, 5])
dat2 = np.random.randint(20, size=[3, 5])
data = DataChunkIterator(iter_datasetvieww(dat1))
data2 = DataChunkIterator(iter_datasetvieww(dat2))
# data = np.array([[0., 1., 2., 3., 4., 5., 6., 7., 8., 9.], [0., 1., 2., 3., 4., 5., 6., 7., 8., 9.]])
# data2 = np.array([[0., 1., 2., 3., 4., 5., 6., 7., 8., 9.], [0., 1., 2., 3., 4., 5., 6., 7., 8., 9.]])
timestamps = np.array([0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])
# data = np.array([data, data]).T
# timestamps1=np.array([timestamps,timestamps])
rrs = fl.create_roi_response_series('my_rrs', data, rt_region, unit='lumens', timestamps=timestamps)
rrs1 = fl.create_roi_response_series(
    'my_rrs1', data2, rt_region1, unit='lumens', timestamps=timestamps)

# ==================================================================================
with NWBHDF5IO('ophys_example_test.nwb', 'w') as io:
    io.write(nwbfile)
io = NWBHDF5IO('ophys_example_test.nwb', 'r')
nwbfile = io.read()
# ==================================================================================

mod = nwbfile.processing['ophys']

ps = mod['ImageSegmentation'].get_plane_segmentation('my_planeseg')


# img_mask1 = ps['image_mask'][0]
# pix_mask1 = ps['pixel_mask'][0]
# img_mask2 = ps['image_mask'][1]
# pix_mask2 = ps['pixel_mask'][1]

rrs = mod['Fluorescence'].get_roi_response_series('my_rrs')


# get the data...
rrs_data = np.array(rrs.data)
data_match = np.all(rrs_data == dat1[:])
if data_match:
    print("Success: All data values match")
else:
    print("ERROR: Mismatch between data")
rrs_timestamps = rrs.timestamps
rrs_rois = rrs.rois
# and now do something cool!

# io.close()


NameError: name 'mod' is not defined